In [1]:
%cd ~/NetBeansProjects/ExpLosion/
from notebooks.common_imports import *
from gui.output_utils import *

query = {
#         'document_features_ev': 'AN+NN', # there are ones with VPs too
#          'document_features_tr': 'J+N+AN+NN',
         'expansions__decode_handler': 'SignifiedOnlyFeatureHandler',
         'expansions__vectors__dimensionality': 100,
         'expansions__vectors__rep': 0,
         'expansions__vectors__unlabelled': 'turian'}

/Volumes/LocalDataHD/m/mm/mmb28/NetBeansProjects/ExpLosion


In [2]:
ids = Experiment.objects.filter(**query).values_list('id', flat=True)
print('ids are', ids)
df = dataframe_from_exp_ids(ids, {'Algorithm':'expansions__vectors__algorithm', 
                                  'Composer':'expansions__vectors__composer',
                                 'Features': 'document_features_tr'})
grid = sns.factorplot(hue='Algorithm', y='Accuracy', data=df, 
                      x='Composer', col='Features',
                      kind='bar', legend=False);
grid.set_xticklabels(rotation=60);
for ax in grid.axes.flat:
    ax.axhline(nondistributional_baseline(), c='k')
    ax.axhline(random_vect_baseline(), c='k')
plt.savefig('plot-turian-different-composers.pdf', format='pdf', dpi=300)

ids are [19, 20, 21, 22, 23]


DoesNotExist: Results matching query does not exist.

In [3]:
df.head()

,Accuracy,Algorithm,Composer,Features,folds
0,0.197970,turian,Add,J+N+AN+NN,0
1,0.199463,turian,Add,J+N+AN+NN,1
2,0.200358,turian,Add,J+N+AN+NN,2
3,0.201851,turian,Add,J+N+AN+NN,3
4,0.202448,turian,Add,J+N+AN+NN,4


In [35]:
def f1(x):
    return '%1.2f' % x

# ddf = df.drop('folds', axis=1).groupby(['Composer', 'k']).agg([np.mean, np.std])
# ddf.columns = ddf.columns.droplevel(0)#.reset_index()
# ddf['Accuracy'] = ddf['mean'].map(f1) + "$\pm$" + ddf['std'].map(f1)
# ddf = ddf.drop(['mean', 'std'], axis=1).reset_index()
# print(ddf.pivot_table(values='Accuracy', index='k', 
#                       columns='Composer', aggfunc=lambda x: x).to_latex(escape=False))

ddf = df.drop(['folds', 'Algorithm'], axis=1).groupby(['Composer', 'Features']).agg('mean').reset_index() # no need to drop unwanted columns
res = ddf.pivot_table(values='Accuracy', index='Composer', columns='Features')
print(res.to_latex(float_format=f1, na_rep='N/A'))
res.T

\begin{tabular}{lrr}
\toprule
Features &  J+N+AN+NN &  J+N+V \\
\midrule
Composer &            &        \\
Add      &       0.22 &   0.75 \\
Left     &       0.23 &    nan \\
Mult     &       0.23 &   0.72 \\
Right    &       0.21 &    nan \\
Socher   &       0.21 &    nan \\
Verb     &        nan &   0.73 \\
\bottomrule
\end{tabular}



Composer,Add,Left,Mult,Right,Socher,Verb
Features,,,,,,
J+N+AN+NN,0.219473,0.228086,0.227260,0.211679,0.210561,NaN
J+N+V,0.749560,NaN,0.724322,NaN,NaN,0.733057


In [36]:
del res.index.name
del res.columns.name
for c in res.columns:
    print(res[[c]].to_latex(float_format=f1, na_rep='N/A'))
res[[c]]

\begin{tabular}{lr}
\toprule
{} &  J+N+AN+NN \\
\midrule
Add    &       0.22 \\
Left   &       0.23 \\
Mult   &       0.23 \\
Right  &       0.21 \\
Socher &       0.21 \\
Verb   &        nan \\
\bottomrule
\end{tabular}

\begin{tabular}{lr}
\toprule
{} &  J+N+V \\
\midrule
Add    &   0.75 \\
Left   &    nan \\
Mult   &   0.72 \\
Right  &    nan \\
Socher &    nan \\
Verb   &   0.73 \\
\bottomrule
\end{tabular}



,J+N+V
Add,0.749560
Left,NaN
Mult,0.724322
Right,NaN
Socher,NaN
Verb,0.733057


In [15]:
ddf.index

Int64Index([0, 1, 2, 3, 4, 5, 6, 7], dtype='int64')

In [19]:
del res.index.name
res

Features,J+N+AN+NN,J+N+V
Add,0.219473,0.749560
Left,0.228086,NaN
Mult,0.227260,0.724322
Right,0.211679,NaN
Socher,0.210561,NaN
Verb,NaN,0.733057


In [21]:
del res.columns.name
res

,J+N+AN+NN,J+N+V
Add,0.219473,0.749560
Left,0.228086,NaN
Mult,0.227260,0.724322
Right,0.211679,NaN
Socher,0.210561,NaN
Verb,NaN,0.733057
